<a href="https://colab.research.google.com/github/smbeche/smbeche.github.io/blob/main/colab/database_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Database Architecture

In the lecture we learned, that there is _no free lunch_ and different databases are designed for different purposes and therefore have advantages and disadvantages. In this python notebook, we want to gather some practical experiences
using databases.

## Indices in Action

### Background Story

A fictional company *QuickMart* is a rapidly growing online store for electronics with thousands of products across various categories. With a vast customer base, QuickMart relies on fast and efficient product searches to keep up with users’ demands and to ensure low latencies. Every time a customer searches for an item, such as “Bluetooth Headphones,” the system needs to retrieve matching products from thousands of entries in real-time.

For simplicity, QuickMart’s search function uses exact product names, making it critical to match customer queries to the Product Title column in the database. However, with a growing catalog, finding an exact match by sequentially scanning every product title is increasingly time-consuming. Developers from QuickMart reached out to you, since you are an expert in optimizing database accesses.

### Overview
1. **Naive Approach**: We are starting slowly and implementing some functions which simulates the naive approach to access the products in a database. This is how Quickmart did up to now. Since we are already familiar with _pandas_, we are using pandas as the database of our choice to apply our operations.
2. **Quick Access using the simulation of an index**: Using the naive approach, we have gained a feeling about efficiency and up to how many requests this might scale. However, Quickmart grew rapidly in the past, therefore we need another strategy for the access into the database. Here we will simulate the index access to get an impression about what indices are capable of.

### 1. Naive Approach

In [3]:
import pandas as pd

url = "https://raw.githubusercontent.com/styx0r/msb_lecture_data_management_and_business_intelligence_2025/refs/heads/main/data/pricerunner_aggregate.csv"
products = pd.read_csv(url)

def get_product(product_title, products):
    # Given a product_title and the products, please find the row,
    # where the product_title equals the column 'Product Title' in products
    return products[products['Product Title'] == product_title]


In [4]:
%%time

# Given the get_product function, the product and a product_title,
# how long does it take to get the information of a product?
# Note: we can use the cell magic function %%time to measure the time of the execution of this cell.

# Thing about what the database is doing in order to return one product. Is it efficient?

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [5]:
# In the backend of Quickmart, the get_product function is called not just once,
# but hundreds of times per second. So next we want to measure the
# time it takes to call the process k (e.g. 2000) times.
# So first of all we need to randomly select k products and save them to a list.

k = 2000

# Please select randomly k elements from the 'Product Title' column of products, which
# which we will user later to access the products k times.
# Since products can be accessed multiple time, please sample k products randomly
# with replacement.


In [6]:
%%time

# Here we want to measure the time when we call the get_products function with
# our samples k random products.

# Please iteratively use the get_product function to get all products from random_product_titles.
# Hint: You can use a for loop. Alternatively there is an apply function which you might use.

# How long does it take for k = 2000?
# How long does it take for 1000, for 2000, ... ? How does it scale regarding time?

# Hint: When asking about scaling, we want to find out how much is the increase of a unit (e.g. time)
# when the complexity of the problem increases (e.g. k).
# Imagine you plot k against the time. If it is a straight line, it's called a linear scale.
# If for example it is a logarithmic or an exponential function it is a log scale or exponential scale, respectively.

# To solve this task, we need to change the k from the cell before and run this cell and remember the times it took.

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [7]:
import time
import matplotlib.pyplot as plt

# Additional Task: Automate the process we just did manually.
# Please run it for incremental sample sizes k and create a plot.
# The x axis should be the sample size and the y axis the time it took to get the products.

# Hint: to measure time between lines of code in python, you can use the
# time library. After importing time, time.process_time() returns a timestamp
# in seconds at this very point in time.

# Define the different sample sizes (k values) you want to test

# Does the plot support your thoughts regarding the scale?


### 2. Quick Access using the simulation of an index

Quickmart has grown a lot recently. They have about 2000 product requests every second. They realized that the shop got really slow and customers started to complain. They have the feeling that they could grow even faster but customers are annoyed of the slow shop and therefore prefer to buy electronics somewhere else. What should they do? You remember a very nice module called 'BI and Data Management' during your masters and there you've heard about indices and that those might help to improve the reading times when accessing the database. So let's get to work...

You remember, the first ingredient is the fast access of a database if we use the row number / index. In pandas we can do that by using loc. E.g. to get the 42th entry of the datatable we can call products.loc[42].
It's also possible to access multiple row numbers at once by using an array, e.g. like this: products.loc[[23,42]]

The second ingredient that we need is an auxiliary table which allows us to efficiently
get the row numbers of the products table given a product title. In the lecture we discussed the binary search.
However, in this exercise we would like to use a dictionary from python. In a python dictionary we have the possible to very efficiently map a given string to a data type of your choice. In our case, we just want to map the product titles to the row numbers.

In [8]:
# Please create the auxiliary table, which can be used to efficiently access the original products table.

In [9]:
# Create a dictionary mapping Product Title to its index in the products DataFrame
product_index_map = dict(zip(products['Product Title'], products.index))

In [10]:
def get_product_fast(product_title, products, index_map):
    # Use the index_map to find the row index for the product_title
    # Then use .loc to retrieve the row from the products DataFrame
    idx = index_map.get(product_title)
    if idx is not null:
        return products.loc[[idx]]
    return pd.DataFrame()

In [11]:
# Now it's the moment of truth, was our effort worth it? Let's measure the time for 10000 hits!
k = 10000

# So let's create the random product titles again.
random_product_titles = products['Product Title'].sample(n=k, replace=True).tolist()

In [12]:
%%time

# Measure the time!

# How long does it take for 1000, 10000, 100000, ...? How does it scale regarding time?

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [13]:
import time
import matplotlib.pyplot as plt

# Additional Task: Again, automate the process we just did manually.
# Please run it for incremental sample sizes k and create a plot.
# The x axis should be the sample size and the y axis the time it took to get the products.

# Hint: to measure time between lines of code in python, you can use the
# time library. After importing time, time.process_time() returns a timestamp
# in seconds at this very point in time.

# Define the different sample sizes (k values) you want to test

# Does the plot support your thoughts regarding the scale?
# Approximately how many product requests can be processed in one second?


We made it! We solved the issue regarding product access times for Quickmart. You can now make an index proposal with your head held high and put it into production. However, this is beyond our exercise ;)

### volunteer Homework

Quickmart has noted that not only with the increased number of customers and requests into the product table everything got slower, but also with the number of products they have in the database. They didn't put a lot of effort to remove discontinued products from the products table. Now they have asked themselves whether it is possible to estimate how much slower the system becomes with an increasing number of products for the slow and fast variants.

To answer this question, we need to fix the number of requests into the data table (.e.g. 10000 for the slow access an 200000 for the quick one) and vary the number of products from the products table. Please try to estimate the influence (the scaling regarding time) when varying the size of the products table (e.g. 1000, 5000, 10000, 15000, 20000, 25000, 30000, 35000).

Additional Task: Provide a plot with the x-axis corresponding to the number of products and the y axis corresponding to the time it took for the request of products. How does time scale with an increased number of products?

## SQL for Beginners

In pandas we can also use SQL to query data. In this exercise, we want to try out a few basic SQL operations to get an idea of its power. To do that, we download the data from the pandas visualization exercise and we are all setup to run some queries.

In [14]:
# installing the missing package pandasql in colab
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=25dcee4f51efff43cece7cc7f23ab5fe9f159a79ed6a2ce1dca48e2f3d9cb319
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql


In [15]:
# create function to download the dataframe

# import packages
import pandas as pd
import requests
import matplotlib
# for animation
matplotlib.rcParams['animation.embed_limit'] = 50.0

# urls
URL_LIFE_EXPECTANCY = 'https://small-waffle.gapminder.org/fasttrack/master/b2642a8?_select_key@=country&=time;&value@=lex;;&from=datapoints&where_'
URL_GDP = 'https://small-waffle.gapminder.org/fasttrack/master/b2642a8?_select_key@=country&=time;&value@=gdp/_pcap;;&from=datapoints&where_'
URL_POPULATION = 'https://small-waffle.gapminder.org/fasttrack/master/b2642a8?_select_key@=country&=time;&value@=pop;;&from=datapoints&where_'
URL_COUNTRY = 'https://restcountries.com/v3.1/all?fields=name,cca3,region'

def read_url(url):
    data = requests.get(url).json()
    return pd.DataFrame(data['rows'], columns=data['header'])

def retrieve_country_information():
    response = requests.get(URL_COUNTRY)
    cs = pd.DataFrame(response.json())[['name', 'cca3', 'region']]
    country = cs.name.map(lambda x: x['common'])
    cs = cs.assign(country=country)
    cs = cs.drop(columns='name')
    cs = cs.rename(columns={'cca3': 'country_short'})
    cs = cs.assign(country_short = cs.country_short.str.lower())
    return cs

def get_data():
    keys = ['country', 'time']
    df_life_expectancy = read_url(URL_LIFE_EXPECTANCY)
    df_gdp = read_url(URL_GDP)
    df_population = read_url(URL_POPULATION)
    df_countries = retrieve_country_information()
    df = (df_life_expectancy
          .merge(df_gdp, on=keys)
          .merge(df_population, on=keys)
          .rename(columns={'country': 'country_short'})
          .merge(df_countries, on='country_short', how='left')
        [['time', 'country', 'region', 'gdp_pcap', 'pop', 'lex']])

    return df

In [16]:
from pandasql import sqldf

data = get_data()

In [17]:
# task 1: display the first ten rows from the pandas dataframe 'data' using pandas
display(data.head(10))

,time,country,region,gdp_pcap,pop,lex
0,1800,Afghanistan,Asia,560.88817,3280000,28.21
1,1801,Afghanistan,Asia,560.88817,3280000,28.20
2,1802,Afghanistan,Asia,560.88817,3280000,28.19
3,1803,Afghanistan,Asia,560.88817,3280000,28.18
4,1804,Afghanistan,Asia,560.88817,3280000,28.17
5,1805,Afghanistan,Asia,560.88817,3280000,28.16
6,1806,Afghanistan,Asia,560.88817,3280000,28.15
7,1807,Afghanistan,Asia,560.88817,3280000,28.14
8,1808,Afghanistan,Asia,560.88817,3280000,28.13
9,1809,Afghanistan,Asia,560.88817,3280000,28.12


In [18]:
# task 2: Using SQL, try to display the same table
# hint: you can call an SQL statement using sqldf("SQL QUERY HERE")

In [19]:
# task 2: Using SQL, try to display the same table
# hint: you can call an SQL statement using sqldf("SQL QUERY HERE")
query = "SELECT * FROM data LIMIT 10"
sqldf(query)

,time,country,region,gdp_pcap,pop,lex
0,1800,Afghanistan,Asia,560.88817,3280000,28.21
1,1801,Afghanistan,Asia,560.88817,3280000,28.20
2,1802,Afghanistan,Asia,560.88817,3280000,28.19
3,1803,Afghanistan,Asia,560.88817,3280000,28.18
4,1804,Afghanistan,Asia,560.88817,3280000,28.17
5,1805,Afghanistan,Asia,560.88817,3280000,28.16
6,1806,Afghanistan,Asia,560.88817,3280000,28.15
7,1807,Afghanistan,Asia,560.88817,3280000,28.14
8,1808,Afghanistan,Asia,560.88817,3280000,28.13
9,1809,Afghanistan,Asia,560.88817,3280000,28.12


In [24]:
# task 3
# Using SQL, how many rows do we have?
query = "SELECT COUNT(*) FROM data"
sqldf(query)

,COUNT(*)
0,57345


In [25]:
# task 4
# Using SQL, how many rows do we have when filtering out the predictions (time > 2024)
query = "SELECT COUNT(*) FROM data WHERE time <= 2024"
sqldf(query)

,COUNT(*)
0,42525


In [26]:
# task 5
# Using SQL, what is the number of countries within a region?
query = "SELECT region, COUNT(country) FROM data GROUP BY region"
sqldf(query)

,region,COUNT(country)
0,Africa,16254
1,Americas,10235
2,Asia,14749
3,Europe,12493
4,Oceania,3614


In [27]:
# task 6
# Did you realize that the number of countries in a region is quite high? Let's refine the task:
# Using SQL, what is the number of distinct countries within a region?
query = "SELECT region, COUNT(DISTINCT country) FROM data GROUP BY region"
sqldf(query)

,region,COUNT(DISTINCT country)
0,Africa,54
1,Americas,35
2,Asia,49
3,Europe,43
4,Oceania,14
